# Data Augmentation

In [7]:
import sys, os, logging
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
def single_graph(df):
    fig = go.Figure()
    for col in df.columns:
      fig.add_trace(go.Scatter(x=df.index,y=df[col],name=col))
    return fig

def triple_graph(df):
    col_num = 1
    fig = make_subplots(rows=1, cols=3)
    for col in df.columns:
      fig.add_trace(go.Scatter(x=df.index,y=df[col],name=col), row=1, col=col_num)
      col_num += 1
    fig.update_layout(height=1080/2, width=1920/2, title_text="Test")
    return fig

In [9]:
DATE = "2023_04_14"
DATA_FOLDER = os.path.join("../data/android_app/", DATE)

DATA_FILES = [file for file in os.listdir(DATA_FOLDER) if os.path.isfile(os.path.join(DATA_FOLDER, file)) and file.endswith(".txt")]

SEPARATOR = ";"

In [10]:
# Acceleration data
data = pd.read_csv(os.path.join(DATA_FOLDER, DATA_FILES[0]), sep=SEPARATOR, header=1, names=["ay", "ax", "az"])

# acc_data = pd.read_csv(os.path.join(DATA_FOLDER, "DATA" + str(REC) + ".txt"), sep=";", header=1, names=["t", "ax", "ay", "az", "loc"])

## Revert Sequence

In [11]:
fig = single_graph(data[["ax", "ay", "az"]])
fig.show()

In [12]:
revert_data = data[::-1].reset_index(drop=True)

In [13]:
fig = single_graph(revert_data[["ax", "ay", "az"]])
fig.show()

In [14]:
def revertSequence(folder, file):
    data = pd.read_csv(os.path.join(folder, file), sep=";", header=1, names=["ay", "ax", "az"])
    revert_data = data[::-1].reset_index(drop=True)
    revert_data.to_csv(os.path.join(folder, file[:len(file)-4] + "_r.txt"), sep=";")

In [15]:
def revertSequences(folder):
    for file in [file for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file)) and file.endswith(".txt")]:
        revertSequence(folder, file)

In [16]:
# Create files
# revertSequences(DATA_FOLDER)

In [17]:
# Remove files
# for file in [file for file in os.listdir(DATA_FOLDER) if os.path.isfile(os.path.join(DATA_FOLDER, file)) and file.endswith("_r.txt")]:
#     os.remove(os.path.join(DATA_FOLDER, file))

## Invert X-axis

In [18]:
fig = single_graph(data[["ax", "ay", "az"]])
fig.show()

In [19]:
inverted_data = data
inverted_data["ax"] = inverted_data["ax"].apply(lambda x: -x)

In [20]:
fig = single_graph(inverted_data[["ax", "ay", "az"]])
fig.show()

In [25]:
def invertSequenceX(folder, file):
    data = pd.read_csv(os.path.join(folder, file), sep=";", header=1, names=["ay", "ax", "az"])
    data["ax"] = data["ax"].apply(lambda x: -x)
    data.to_csv(os.path.join(folder, file[:len(file)-4] + "_i.txt"), sep=";")

In [26]:
def invertSequencesX(folder):
    for file in [file for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file)) and file.endswith(".txt")]:
        invertSequenceX(folder, file)

In [27]:
# Create files
invertSequencesX(DATA_FOLDER)

## Add Noise

### Hyperparameters : sigma = standard devitation (STD) of the noise

In [ ]:
sigma = 0.1

In [ ]:
def DA_Jitter(X, sigma):
    myNoise = np.random.normal(loc=0, scale=sigma, size=X.shape)
    return X+myNoise

In [ ]:
def addNoise_1(folder, file):
    data = pd.read_csv(os.path.join(folder, file), sep=";", header=1, names=["ay", "ax", "az"])
    noise = DA_Jitter(data,sigma)
    noise.to_csv(os.path.join(folder, file[:len(file)-4] + "_n.txt"), sep=";")

In [ ]:
def addNoise(folder):
    for file in [file for file in os.listdir(folder) if os.path.isfile(os.path.join(folder, file)) and file.endswith(".txt")]:
        addNoise_1(folder, file)

In [ ]:
# Create files
addNoise(DATA_FOLDER)

In [ ]:
#Remove files
for file in [file for file in os.listdir(DATA_FOLDER) if os.path.isfile(os.path.join(DATA_FOLDER, file)) and file.endswith("_n.txt")]:
     os.remove(os.path.join(DATA_FOLDER, file))

In [ ]:
fig = single_graph(data[["ax", "ay", "az"]])
fig.show()